In [2]:
import os
import pickle
import sys
from argparse import ArgumentParser

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils.data import DataLoader
from torchvision import transforms

from utils import NUM_PTS, CROP_SIZE
from utils import ScaleMinSideToSize, CropCenter, TransformByKeys
from utils import ThousandLandmarksDataset
from utils import restore_landmarks_batch, create_submission

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

from train import train, validate, predict

In [3]:
train_transforms = transforms.Compose([
        ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
        CropCenter(CROP_SIZE),
        TransformByKeys(transforms.ToPILImage(), ("image",)),
        TransformByKeys(transforms.ToTensor(), ("image",)),
        TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]), ("image",)),
    ])

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
test_dataset = ThousandLandmarksDataset(os.path.join('.', "test"), train_transforms, split="test")
test_dataloader = DataLoader(test_dataset, batch_size=512, num_workers=2, pin_memory=True,
                                 shuffle=False, drop_last=False)

100%|██████████| 16001/16001 [00:00<00:00, 354570.26it/s]


In [7]:
test_predictions = predict(model, test_dataloader, device)
with open(os.path.join("runs", f"{'03'}_test_predictions.pkl"), "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                     "landmarks": test_predictions}, fp)

create_submission('.', test_predictions, os.path.join("runs", f"{'resnext101_32x8d_no_noise'}_submit.csv"))

test prediction...: 100%|██████████| 32/32 [33:57<00:00, 63.66s/it]


In [5]:
model = models.resnext101_32x8d(pretrained=True, progress=False)
model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)

In [6]:
name = './runs/resnext101_32x8d_ep24_loss3.223.pth'
with open(f"{name}", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)
# freeze all layers
for param in model.parameters():
    param.requires_grad = False
        #
        # # train only head
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True
        #
        # # train layer4
for param in model.layer4.parameters():
    param.requires_grad = True